In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
adventure=pd.read_csv("adventure.csv")

In [3]:
adventure.head(10)


,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt9114286,Black Panther: Wakanda Forever,2022,PG-13,161 min,"Action, Adventure, Drama",6.9,The people of Wakanda fight to protect their h...,Ryan Coogler,/name/nm3363032/,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri...","/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN
1,tt1630029,Avatar: The Way of Water,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.8,Jake Sully lives with his newfound family form...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN
2,tt6710474,Everything Everywhere All at Once,2022,R,139 min,"Action, Adventure, Comedy",8.0,A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert",/name/nm3453283/,"Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu...","/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN
3,tt3915174,Puss in Boots: The Last Wish,2022,PG,102 min,"Animation, Adventure, Comedy",7.9,When Puss in Boots discovers that his passion ...,"Joel Crawford, \nJanuel Mercado",/name/nm3150455/,"Antonio Banderas, \nSalma Hayek, \nHarvey Guil...","/name/nm2591093/,/name/nm0000104/,/name/nm0000...",75016.0,NaN
4,tt10954600,Ant-Man and the Wasp: Quantumania,2023,PG-13,125 min,"Action, Adventure, Comedy",6.6,"Scott Lang and Hope Van Dyne, along with Hank ...",Peyton Reed,/name/nm0715636/,"Paul Rudd, \nEvangeline Lilly, \nJonathan Majo...","/name/nm0748620/,/name/nm1431940/,/name/nm3718...",5396.0,NaN
5,tt2353868,True Spirit,2023,TV-PG,109 min,"Adventure, Biography, Drama",6.8,"The story of Australian teenager, Jessica Wats...",Sarah Spillane,/name/nm1417049/,"Alyla Browne, \nTeagan Croft, \nCliff Curtis, ...","/name/nm10752061/,/name/nm7509279/,/name/nm019...",3747.0,NaN
6,tt12844910,Pathaan,2023,Not Rated,146 min,"Action, Adventure, Drama",6.6,An Indian spy takes on the leader of a group o...,Siddharth Anand,/name/nm1893457/,"Shah Rukh Khan, \nDeepika Padukone, \nJohn Abr...","/name/nm0451321/,/name/nm2138653/,/name/nm1303...",120080.0,NaN
7,tt14668630,"Lyle, Lyle, Crocodile",2022,PG,106 min,"Animation, Adventure, Comedy",6.1,Feature film based on the children's book abou...,"Josh Gordon, \nWill Speck",/name/nm0330347/,"Javier Bardem, \nWinslow Fegley, \nShawn Mende...","/name/nm0817447/,/name/nm0000849/,/name/nm9121...",8131.0,NaN
8,tt1825683,Black Panther,2018,PG-13,134 min,"Action, Adventure, Sci-Fi",7.3,"T'Challa, heir to the hidden but advanced king...",Ryan Coogler,/name/nm3363032/,"Chadwick Boseman, \nMichael B. Jordan, \nLupit...","/name/nm1569276/,/name/nm0430107/,/name/nm2143...",785813.0,700059566.0
9,tt0499549,Avatar,2009,PG-13,162 min,"Action, Adventure, Fantasy",7.9,A paraplegic Marine dispatched to the moon Pan...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",1322694.0,760507625.0


In [4]:
adventure.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25664 entries, 0 to 25663
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     25664 non-null  object 
 1   movie_name   25664 non-null  object 
 2   year         21975 non-null  object 
 3   certificate  8080 non-null   object 
 4   runtime      18450 non-null  object 
 5   genre        25664 non-null  object 
 6   rating       17082 non-null  float64
 7   description  25664 non-null  object 
 8   director     23662 non-null  object 
 9   director_id  23662 non-null  object 
 10  star         21722 non-null  object 
 11  star_id      22126 non-null  object 
 12  votes        17083 non-null  float64
 13  gross(in $)  2088 non-null   float64
dtypes: float64(3), object(11)
memory usage: 2.7+ MB


In [5]:
animation=pd.read_csv("animation.csv")

In [6]:
animation.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8419 entries, 0 to 8418
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     8419 non-null   object 
 1   movie_name   8419 non-null   object 
 2   year         7050 non-null   object 
 3   certificate  2087 non-null   object 
 4   runtime      5907 non-null   object 
 5   genre        8419 non-null   object 
 6   rating       4761 non-null   float64
 7   description  8419 non-null   object 
 8   director     7517 non-null   object 
 9   director_id  7517 non-null   object 
 10  star         5570 non-null   object 
 11  star_id      6059 non-null   object 
 12  votes        4761 non-null   float64
 13  gross(in $)  574 non-null    float64
dtypes: float64(3), object(11)
memory usage: 921.0+ KB


In [7]:
animation.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt3915174,Puss in Boots: The Last Wish,2022,PG,102 min,"Animation, Adventure, Comedy",7.9,When Puss in Boots discovers that his passion ...,"Joel Crawford, \nJanuel Mercado",/name/nm3150455/,"Antonio Banderas, \nSalma Hayek, \nHarvey Guil...","/name/nm2591093/,/name/nm0000104/,/name/nm0000...",93143.0,168464485.0
1,tt6718170,The Super Mario Bros. Movie,2023,PG,92 min,"Animation, Adventure, Comedy",NaN,The story of The Super Mario Bros. on their jo...,"Aaron Horvath, \nMichael Jelenic",/name/nm1739338/,"Chris Pratt, \nAnya Taylor-Joy, \nCharlie Day,...","/name/nm2398585/,/name/nm0695435/,/name/nm5896...",NaN,NaN
2,tt26537229,Demon Slayer: Kimetsu No Yaiba - To the Swords...,2023,R,110 min,"Animation, Action, Adventure",6.6,All the Upper Rank Demons assemble at the Infi...,Haruo Sotozaki,/name/nm1417038/,"Zach Aguilar, \nKira Buckland, \nGriffin Burns...","/name/nm6450743/,/name/nm2299231/,/name/nm4232...",1316.0,NaN
3,tt1488589,Guillermo del Toro's Pinocchio,2022,PG,117 min,"Animation, Drama, Family",7.6,A father's wish magically brings a wooden boy ...,"Guillermo del Toro, \nMark Gustafson",/name/nm0868219/,"Ewan McGregor, \nDavid Bradley, \nGregory Mann...","/name/nm0348993/,/name/nm0000191/,/name/nm0103...",86296.0,NaN
4,tt14668630,"Lyle, Lyle, Crocodile",2022,PG,106 min,"Animation, Adventure, Comedy",6.1,Feature film based on the children's book abou...,"Josh Gordon, \nWill Speck",/name/nm0330347/,"Javier Bardem, \nWinslow Fegley, \nShawn Mende...","/name/nm0817447/,/name/nm0000849/,/name/nm9121...",9351.0,46888441.0


In [8]:
biography=pd.read_csv("biography.csv")


In [9]:
biography.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8289 entries, 0 to 8288
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     8289 non-null   object 
 1   movie_name   8289 non-null   object 
 2   year         6284 non-null   object 
 3   certificate  1988 non-null   object 
 4   runtime      5171 non-null   object 
 5   genre        8289 non-null   object 
 6   rating       4380 non-null   float64
 7   description  8289 non-null   object 
 8   director     7054 non-null   object 
 9   director_id  7054 non-null   object 
 10  star         6045 non-null   object 
 11  star_id      6161 non-null   object 
 12  votes        4380 non-null   float64
 13  gross(in $)  881 non-null    float64
dtypes: float64(3), object(11)
memory usage: 906.7+ KB


In [10]:
biography.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt3704428,Elvis,2022,PG-13,159 min,"Biography, Drama, Music",7.3,"The life of American music icon Elvis Presley,...",Baz Luhrmann,/name/nm0525303/,"Tom Hanks, \nAustin Butler, \nOlivia DeJonge, ...","/name/nm0000158/,/name/nm2581521/,/name/nm4609...",189732.0,151040048.0
1,tt0993846,The Wolf of Wall Street,2013,R,180 min,"Biography, Comedy, Crime",8.2,"Based on the true story of Jordan Belfort, fro...",Martin Scorsese,/name/nm0000217/,"Leonardo DiCaprio, \nJonah Hill, \nMargot Robb...","/name/nm0000138/,/name/nm1706767/,/name/nm3053...",1437780.0,116900694.0
2,tt0264464,Catch Me If You Can,2002,PG-13,141 min,"Biography, Crime, Drama",8.1,"Barely 21 yet, Frank is a skilled forger who h...",Steven Spielberg,/name/nm0000229/,"Leonardo DiCaprio, \nTom Hanks, \nChristopher ...","/name/nm0000138/,/name/nm0000158/,/name/nm0000...",1003050.0,164615351.0
3,tt12374656,Emily,2022,R,130 min,"Biography, Drama, Romance",6.9,"""Emily"" imagines the transformative, exhilarat...",Frances O'Connor,/name/nm0640323/,"Emma Mackey, \nOliver Jackson-Cohen, \nFionn W...","/name/nm8076281/,/name/nm2719825/,/name/nm7887...",3888.0,NaN
4,tt15398776,Oppenheimer,2023,NaN,NaN,"Biography, Drama, History",NaN,The story of American scientist J. Robert Oppe...,Christopher Nolan,/name/nm0634240/,"Cillian Murphy, \nEmily Blunt, \nMatt Damon, \...","/name/nm0614165/,/name/nm1289434/,/name/nm0000...",NaN,NaN


In [11]:
action=pd.read_csv("action.csv")

In [12]:
action.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52452 entries, 0 to 52451
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     52452 non-null  object 
 1   movie_name   52451 non-null  object 
 2   year         44193 non-null  object 
 3   certificate  13668 non-null  object 
 4   runtime      32530 non-null  object 
 5   genre        52452 non-null  object 
 6   rating       29448 non-null  float64
 7   description  52452 non-null  object 
 8   director     47819 non-null  object 
 9   director_id  47819 non-null  object 
 10  star         43212 non-null  object 
 11  star_id      44621 non-null  object 
 12  votes        29450 non-null  float64
 13  gross(in $)  2760 non-null   float64
dtypes: float64(3), object(11)
memory usage: 5.6+ MB


In [13]:
action.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt9114286,Black Panther: Wakanda Forever,2022,PG-13,161 min,"Action, Adventure, Drama",6.9,The people of Wakanda fight to protect their h...,Ryan Coogler,/name/nm3363032/,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri...","/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN
1,tt1630029,Avatar: The Way of Water,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.8,Jake Sully lives with his newfound family form...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN
2,tt5884796,Plane,2023,R,107 min,"Action, Thriller",6.5,A pilot finds himself caught in a war zone aft...,Jean-François Richet,/name/nm0724938/,"Gerard Butler, \nMike Colter, \nTony Goldwyn, ...","/name/nm0124930/,/name/nm1591496/,/name/nm0001...",26220.0,NaN
3,tt6710474,Everything Everywhere All at Once,2022,R,139 min,"Action, Adventure, Comedy",8.0,A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert",/name/nm3453283/,"Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu...","/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN
4,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN


In [14]:
crime=pd.read_csv("crime.csv")

In [15]:
crime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35852 entries, 0 to 35851
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     35852 non-null  object 
 1   movie_name   35852 non-null  object 
 2   year         32289 non-null  object 
 3   certificate  11226 non-null  object 
 4   runtime      27632 non-null  object 
 5   genre        35852 non-null  object 
 6   rating       24340 non-null  float64
 7   description  35852 non-null  object 
 8   director     34251 non-null  object 
 9   director_id  34251 non-null  object 
 10  star         32318 non-null  object 
 11  star_id      32701 non-null  object 
 12  votes        24340 non-null  float64
 13  gross(in $)  2626 non-null   float64
dtypes: float64(3), object(11)
memory usage: 3.8+ MB


In [16]:
crime.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt5433140,Fast X,2023,NaN,NaN,"Action, Crime, Mystery",NaN,Dom Toretto and his family are targeted by the...,Louis Leterrier,/name/nm0504642/,"Vin Diesel, \nJordana Brewster, \nTyrese Gibso...","/name/nm0004874/,/name/nm0108287/,/name/nm0879...",NaN,NaN
1,tt11564570,Glass Onion,2022,PG-13,139 min,"Comedy, Crime, Drama",7.2,Famed Southern detective Benoit Blanc travels ...,Rian Johnson,/name/nm0426059/,"Daniel Craig, \nEdward Norton, \nKate Hudson, ...","/name/nm0185819/,/name/nm0001570/,/name/nm0005...",333315.0,NaN
2,tt10365998,Infinity Pool,2023,R,117 min,"Crime, Horror, Mystery",6.5,James and Em Foster are enjoying an all-inclus...,Brandon Cronenberg,/name/nm0188722/,"Alexander Skarsgård, \nMia Goth, \nCleopatra C...","/name/nm0002907/,/name/nm5301405/,/name/nm1671...",6955.0,NaN
3,tt14138650,The Pale Blue Eye,2022,R,128 min,"Crime, Horror, Mystery",6.6,A world-weary detective is hired to investigat...,Scott Cooper,/name/nm0178376/,"Christian Bale, \nHarry Melling, \nSimon McBur...","/name/nm0000288/,/name/nm0577982/,/name/nm0564...",85087.0,NaN
4,tt15789492,Infiesto,2023,TV-MA,96 min,"Crime, Drama, Thriller",5.5,Two detectives are called to a small mining to...,Patxi Amezcua,/name/nm0025538/,"Isak Férriz, \nIria del Río, \nAntonio Buíl, \...","/name/nm1929945/,/name/nm4170579/,/name/nm0125...",2081.0,NaN


In [17]:
family=pd.read_csv("family.csv")

In [18]:
family.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17095 entries, 0 to 17094
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     17095 non-null  object 
 1   movie_name   17095 non-null  object 
 2   year         15464 non-null  object 
 3   certificate  4613 non-null   object 
 4   runtime      13187 non-null  object 
 5   genre        17095 non-null  object 
 6   rating       11712 non-null  float64
 7   description  17095 non-null  object 
 8   director     16208 non-null  object 
 9   director_id  16208 non-null  object 
 10  star         14952 non-null  object 
 11  star_id      15229 non-null  object 
 12  votes        11712 non-null  float64
 13  gross(in $)  1240 non-null   float64
dtypes: float64(3), object(11)
memory usage: 1.8+ MB


In [19]:
fantasy=pd.read_csv("fantasy.csv")

In [20]:
fantasy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17163 entries, 0 to 17162
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     17163 non-null  object 
 1   movie_name   17163 non-null  object 
 2   year         14349 non-null  object 
 3   certificate  4463 non-null   object 
 4   runtime      12003 non-null  object 
 5   genre        17163 non-null  object 
 6   rating       10447 non-null  float64
 7   description  17163 non-null  object 
 8   director     15728 non-null  object 
 9   director_id  15728 non-null  object 
 10  star         14128 non-null  object 
 11  star_id      14476 non-null  object 
 12  votes        10447 non-null  float64
 13  gross(in $)  1410 non-null   float64
dtypes: float64(3), object(11)
memory usage: 1.8+ MB


In [21]:
horror=pd.read_csv("horror.csv")

In [22]:
horror.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36682 entries, 0 to 36681
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     36682 non-null  object 
 1   movie_name   36682 non-null  object 
 2   year         29091 non-null  object 
 3   certificate  9747 non-null   object 
 4   runtime      23362 non-null  object 
 5   genre        36682 non-null  object 
 6   rating       21159 non-null  float64
 7   description  36682 non-null  object 
 8   director     33617 non-null  object 
 9   director_id  33617 non-null  object 
 10  star         29552 non-null  object 
 11  star_id      30328 non-null  object 
 12  votes        21159 non-null  float64
 13  gross(in $)  1346 non-null   float64
dtypes: float64(3), object(11)
memory usage: 3.9+ MB


In [23]:
mystery=pd.read_csv("mystery.csv")

In [24]:
mystery.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18960 entries, 0 to 18959
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     18960 non-null  object 
 1   movie_name   18959 non-null  object 
 2   year         16956 non-null  object 
 3   certificate  6444 non-null   object 
 4   runtime      15107 non-null  object 
 5   genre        18960 non-null  object 
 6   rating       12921 non-null  float64
 7   description  18960 non-null  object 
 8   director     18229 non-null  object 
 9   director_id  18229 non-null  object 
 10  star         17091 non-null  object 
 11  star_id      17298 non-null  object 
 12  votes        12921 non-null  float64
 13  gross(in $)  1388 non-null   float64
dtypes: float64(3), object(11)
memory usage: 2.0+ MB


In [25]:
romance=pd.read_csv("romance.csv")

In [26]:
romance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52617 entries, 0 to 52616
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie_id     52617 non-null  object 
 1   movie_name   52617 non-null  object 
 2   year         49180 non-null  object 
 3   certificate  16151 non-null  object 
 4   runtime      41073 non-null  object 
 5   genre        52617 non-null  object 
 6   rating       37355 non-null  float64
 7   description  52617 non-null  object 
 8   director     50879 non-null  object 
 9   director_id  50879 non-null  object 
 10  star         48756 non-null  object 
 11  star_id      49427 non-null  object 
 12  votes        37355 non-null  float64
 13  gross(in $)  4075 non-null   float64
dtypes: float64(3), object(11)
memory usage: 5.6+ MB


In [27]:
scifi=pd.read_csv("scifi.csv")

In [28]:
sports=pd.read_csv("sports.csv")

In [29]:
thriller=pd.read_csv("thriller.csv")

In [30]:
war=pd.read_csv("war.csv")

In [31]:
movies=pd.concat([adventure,sports,horror,thriller,mystery,action,scifi,war,fantasy,crime,biography,family],ignore_index=True)

In [32]:
movies.head()

,movie_id,movie_name,year,certificate,runtime,genre,rating,description,director,director_id,star,star_id,votes,gross(in $)
0,tt9114286,Black Panther: Wakanda Forever,2022,PG-13,161 min,"Action, Adventure, Drama",6.9,The people of Wakanda fight to protect their h...,Ryan Coogler,/name/nm3363032/,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri...","/name/nm4004793/,/name/nm2143282/,/name/nm1775...",204835.0,NaN
1,tt1630029,Avatar: The Way of Water,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.8,Jake Sully lives with his newfound family form...,James Cameron,/name/nm0000116/,"Sam Worthington, \nZoe Saldana, \nSigourney We...","/name/nm0941777/,/name/nm0757855/,/name/nm0000...",295119.0,NaN
2,tt6710474,Everything Everywhere All at Once,2022,R,139 min,"Action, Adventure, Comedy",8.0,A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert",/name/nm3453283/,"Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu...","/name/nm3215397/,/name/nm0000706/,/name/nm3513...",327858.0,NaN
3,tt3915174,Puss in Boots: The Last Wish,2022,PG,102 min,"Animation, Adventure, Comedy",7.9,When Puss in Boots discovers that his passion ...,"Joel Crawford, \nJanuel Mercado",/name/nm3150455/,"Antonio Banderas, \nSalma Hayek, \nHarvey Guil...","/name/nm2591093/,/name/nm0000104/,/name/nm0000...",75016.0,NaN
4,tt10954600,Ant-Man and the Wasp: Quantumania,2023,PG-13,125 min,"Action, Adventure, Comedy",6.6,"Scott Lang and Hope Van Dyne, along with Hank ...",Peyton Reed,/name/nm0715636/,"Paul Rudd, \nEvangeline Lilly, \nJonathan Majo...","/name/nm0748620/,/name/nm1431940/,/name/nm3718...",5396.0,NaN


In [33]:
movies.shape

(297282, 14)

In [34]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297282 entries, 0 to 297281
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   movie_id     297282 non-null  object 
 1   movie_name   297278 non-null  object 
 2   year         250135 non-null  object 
 3   certificate  84113 non-null   object 
 4   runtime      204742 non-null  object 
 5   genre        297282 non-null  object 
 6   rating       182335 non-null  float64
 7   description  297282 non-null  object 
 8   director     273315 non-null  object 
 9   director_id  273315 non-null  object 
 10  star         246901 non-null  object 
 11  star_id      252446 non-null  object 
 12  votes        182338 non-null  float64
 13  gross(in $)  19708 non-null   float64
dtypes: float64(3), object(11)
memory usage: 31.8+ MB


In [35]:
rating=movies.loc[movies['movie_name']=='Rajwadi Chhiye Ame Manbher Rahiye','rating'].values[0]
print(rating)

nan


In [36]:
movies['rating'].fillna(0,inplace=True)

In [37]:
movies=movies[movies['rating']>=5]

In [38]:
movies=movies[["movie_name","genre","description","director","star"]]

In [39]:
movies.head()

,movie_name,genre,description,director,star
0,Black Panther: Wakanda Forever,"Action, Adventure, Drama",The people of Wakanda fight to protect their h...,Ryan Coogler,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri..."
1,Avatar: The Way of Water,"Action, Adventure, Fantasy",Jake Sully lives with his newfound family form...,James Cameron,"Sam Worthington, \nZoe Saldana, \nSigourney We..."
2,Everything Everywhere All at Once,"Action, Adventure, Comedy",A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert","Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu..."
3,Puss in Boots: The Last Wish,"Animation, Adventure, Comedy",When Puss in Boots discovers that his passion ...,"Joel Crawford, \nJanuel Mercado","Antonio Banderas, \nSalma Hayek, \nHarvey Guil..."
4,Ant-Man and the Wasp: Quantumania,"Action, Adventure, Comedy","Scott Lang and Hope Van Dyne, along with Hank ...",Peyton Reed,"Paul Rudd, \nEvangeline Lilly, \nJonathan Majo..."


In [40]:
movies.isnull().sum()

movie_name        0
genre             0
description       0
director        137
star           1297
dtype: int64

In [41]:
movies.duplicated().sum()

51528

In [42]:
movies.dropna(inplace=True)

In [43]:
movies.isnull().sum()

movie_name     0
genre          0
description    0
director       0
star           0
dtype: int64

In [44]:
movies.shape

(133730, 5)

In [45]:
movies=movies.drop_duplicates()

In [46]:
movies.shape

(82556, 5)

In [47]:
movies.duplicated().sum()

0

In [48]:
movies.isnull().sum()

movie_name     0
genre          0
description    0
director       0
star           0
dtype: int64

In [49]:
movies=movies.reset_index(drop=True)
movies.head(50000)

,movie_name,genre,description,director,star
0,Black Panther: Wakanda Forever,"Action, Adventure, Drama",The people of Wakanda fight to protect their h...,Ryan Coogler,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri..."
1,Avatar: The Way of Water,"Action, Adventure, Fantasy",Jake Sully lives with his newfound family form...,James Cameron,"Sam Worthington, \nZoe Saldana, \nSigourney We..."
2,Everything Everywhere All at Once,"Action, Adventure, Comedy",A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert","Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu..."
3,Puss in Boots: The Last Wish,"Animation, Adventure, Comedy",When Puss in Boots discovers that his passion ...,"Joel Crawford, \nJanuel Mercado","Antonio Banderas, \nSalma Hayek, \nHarvey Guil..."
4,Ant-Man and the Wasp: Quantumania,"Action, Adventure, Comedy","Scott Lang and Hope Van Dyne, along with Hank ...",Peyton Reed,"Paul Rudd, \nEvangeline Lilly, \nJonathan Majo..."
...,...,...,...,...,...
49995,The Top Bet,"Action, Comedy, Romance",Taiwanese triad leader Chung Chan (Jeffrey Lau...,"Jeffrey Lau, \nCorey Yuen","Carol 'Do Do' Cheng, \nAnita Mui, \nMan-Tat Ng..."
49996,Kunoichi ninpô-chô IV: Chûshingura hishô,"Action, Comedy, Fantasy",The female ninjas have to stop an alliance fro...,Masaru Tsushima,"Tatsuo Endô, \nHisao Hata, \nKenji Naka, \nMeg..."
49997,Sky Devils,"Action, Comedy, Romance","Trying to desert their draft into the army, Wi...",A. Edward Sutherland,"Spencer Tracy, \nWilliam 'Stage' Boyd, \nGeorg..."
49998,The Story of Drunken Master,"Action, Comedy",Beggar So is trying to train brother and siste...,Hai-Feng Wei,"Siu-Tin Yuen, \nPan Pan Yeung, \nHo Wang, \nDe..."


In [50]:
movies.head(51351)

,movie_name,genre,description,director,star
0,Black Panther: Wakanda Forever,"Action, Adventure, Drama",The people of Wakanda fight to protect their h...,Ryan Coogler,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri..."
1,Avatar: The Way of Water,"Action, Adventure, Fantasy",Jake Sully lives with his newfound family form...,James Cameron,"Sam Worthington, \nZoe Saldana, \nSigourney We..."
2,Everything Everywhere All at Once,"Action, Adventure, Comedy",A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert","Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu..."
3,Puss in Boots: The Last Wish,"Animation, Adventure, Comedy",When Puss in Boots discovers that his passion ...,"Joel Crawford, \nJanuel Mercado","Antonio Banderas, \nSalma Hayek, \nHarvey Guil..."
4,Ant-Man and the Wasp: Quantumania,"Action, Adventure, Comedy","Scott Lang and Hope Van Dyne, along with Hank ...",Peyton Reed,"Paul Rudd, \nEvangeline Lilly, \nJonathan Majo..."
...,...,...,...,...,...
51346,Zinda Dil,"Action, Romance",This is the story of two brothers and a father...,Sikandar Khanna,"Rishi Kapoor, \nNeetu Singh, \nZaheera, \nPran..."
51347,She Goes to War,"Action, Drama, War",A young woman disguises herself as a man and f...,Henry King,"Eleanor Boardman, \nJohn Holland, \nEdmund Bur..."
51348,Kalibre .45,Action,THE KALIBRA .45 CALIBER GUN BECOMES A HERO'S I...,Nilo Saez,"Fernando Poe Jr., \nLito Lapid, \nJennifer Cor..."
51349,"The Dragon, the Hero",Action,"John Liu, the legendary Northern Kick expert a...",Godfrey Ho,"John Liu, \nYen-Tsan Tang, \nPhillip Ko, \nLau..."


In [51]:
movies.iloc[9].genre

'Action, Adventure, Fantasy'

In [52]:
movies['genre']=movies['genre'].apply(lambda x : [genre.strip() for genre in x.split(',')])

In [53]:
movies.head(50000)

,movie_name,genre,description,director,star
0,Black Panther: Wakanda Forever,"[Action, Adventure, Drama]",The people of Wakanda fight to protect their h...,Ryan Coogler,"Letitia Wright, \nLupita Nyong'o, \nDanai Guri..."
1,Avatar: The Way of Water,"[Action, Adventure, Fantasy]",Jake Sully lives with his newfound family form...,James Cameron,"Sam Worthington, \nZoe Saldana, \nSigourney We..."
2,Everything Everywhere All at Once,"[Action, Adventure, Comedy]",A middle-aged Chinese immigrant is swept up in...,"Dan Kwan, \nDaniel Scheinert","Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu..."
3,Puss in Boots: The Last Wish,"[Animation, Adventure, Comedy]",When Puss in Boots discovers that his passion ...,"Joel Crawford, \nJanuel Mercado","Antonio Banderas, \nSalma Hayek, \nHarvey Guil..."
4,Ant-Man and the Wasp: Quantumania,"[Action, Adventure, Comedy]","Scott Lang and Hope Van Dyne, along with Hank ...",Peyton Reed,"Paul Rudd, \nEvangeline Lilly, \nJonathan Majo..."
...,...,...,...,...,...
49995,The Top Bet,"[Action, Comedy, Romance]",Taiwanese triad leader Chung Chan (Jeffrey Lau...,"Jeffrey Lau, \nCorey Yuen","Carol 'Do Do' Cheng, \nAnita Mui, \nMan-Tat Ng..."
49996,Kunoichi ninpô-chô IV: Chûshingura hishô,"[Action, Comedy, Fantasy]",The female ninjas have to stop an alliance fro...,Masaru Tsushima,"Tatsuo Endô, \nHisao Hata, \nKenji Naka, \nMeg..."
49997,Sky Devils,"[Action, Comedy, Romance]","Trying to desert their draft into the army, Wi...",A. Edward Sutherland,"Spencer Tracy, \nWilliam 'Stage' Boyd, \nGeorg..."
49998,The Story of Drunken Master,"[Action, Comedy]",Beggar So is trying to train brother and siste...,Hai-Feng Wei,"Siu-Tin Yuen, \nPan Pan Yeung, \nHo Wang, \nDe..."


In [54]:
movies['director']=movies['director'].apply(lambda x : [director.strip() for director in x.split(',')])

In [55]:
movies.head()

,movie_name,genre,description,director,star
0,Black Panther: Wakanda Forever,"[Action, Adventure, Drama]",The people of Wakanda fight to protect their h...,[Ryan Coogler],"Letitia Wright, \nLupita Nyong'o, \nDanai Guri..."
1,Avatar: The Way of Water,"[Action, Adventure, Fantasy]",Jake Sully lives with his newfound family form...,[James Cameron],"Sam Worthington, \nZoe Saldana, \nSigourney We..."
2,Everything Everywhere All at Once,"[Action, Adventure, Comedy]",A middle-aged Chinese immigrant is swept up in...,"[Dan Kwan, Daniel Scheinert]","Michelle Yeoh, \nStephanie Hsu, \nJamie Lee Cu..."
3,Puss in Boots: The Last Wish,"[Animation, Adventure, Comedy]",When Puss in Boots discovers that his passion ...,"[Joel Crawford, Januel Mercado]","Antonio Banderas, \nSalma Hayek, \nHarvey Guil..."
4,Ant-Man and the Wasp: Quantumania,"[Action, Adventure, Comedy]","Scott Lang and Hope Van Dyne, along with Hank ...",[Peyton Reed],"Paul Rudd, \nEvangeline Lilly, \nJonathan Majo..."


In [56]:
movies['star'][0]

"Letitia Wright, \nLupita Nyong'o, \nDanai Gurira, \nWinston Duke"

In [57]:
movies['star']=movies['star'].str.replace("\n"," ")


In [58]:
movies['star'][0]

"Letitia Wright,  Lupita Nyong'o,  Danai Gurira,  Winston Duke"

In [59]:
def clean_cast(cast_str):
  cast_members=cast_str.split(',')[:3]
  cast_members=[member.strip().replace(' " ',' ') for member in cast_members]
  return cast_members
movies['star']=movies['star'].apply(clean_cast)
  

In [60]:
movies['star'][0]

['Letitia Wright', "Lupita Nyong'o", 'Danai Gurira']

In [61]:
movies['star'][0]

['Letitia Wright', "Lupita Nyong'o", 'Danai Gurira']

In [62]:
movies.head()

,movie_name,genre,description,director,star
0,Black Panther: Wakanda Forever,"[Action, Adventure, Drama]",The people of Wakanda fight to protect their h...,[Ryan Coogler],"[Letitia Wright, Lupita Nyong'o, Danai Gurira]"
1,Avatar: The Way of Water,"[Action, Adventure, Fantasy]",Jake Sully lives with his newfound family form...,[James Cameron],"[Sam Worthington, Zoe Saldana, Sigourney Weaver]"
2,Everything Everywhere All at Once,"[Action, Adventure, Comedy]",A middle-aged Chinese immigrant is swept up in...,"[Dan Kwan, Daniel Scheinert]","[Michelle Yeoh, Stephanie Hsu, Jamie Lee Curtis]"
3,Puss in Boots: The Last Wish,"[Animation, Adventure, Comedy]",When Puss in Boots discovers that his passion ...,"[Joel Crawford, Januel Mercado]","[Antonio Banderas, Salma Hayek, Harvey Guillén]"
4,Ant-Man and the Wasp: Quantumania,"[Action, Adventure, Comedy]","Scott Lang and Hope Van Dyne, along with Hank ...",[Peyton Reed],"[Paul Rudd, Evangeline Lilly, Jonathan Majors]"


In [63]:
movies['star'][0]

['Letitia Wright', "Lupita Nyong'o", 'Danai Gurira']

In [64]:
movies[movies['movie_name']==']['genre']

SyntaxError: unterminated string literal (detected at line 1) (2855243634.py, line 1)

In [ ]:
movies['description']=movies['description'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['genre']=movies['genre'].apply(lambda x:[i.replace(" ","")for i in x])
movies['star']=movies['star'].apply(lambda x:[i.replace(" ","")for i in x])
movies['director']=movies['director'].apply(lambda x:[i.replace(" ","")for i in x])


In [ ]:
movies.head()

In [ ]:
movies['tags']=movies['description']+movies['genre']+movies['director']+movies['star']

In [ ]:
movies.head()

In [ ]:
new_df=movies[['movie_name','tags']]

In [ ]:
new_df.head()

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
new_df.shape

In [ ]:
ps=PorterStemmer()
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
 new_df['tags']=new_df['tags'].apply(stem)

In [ ]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

In [ ]:
vectors.shape

In [ ]:
cv.get_feature_names_out()

In [ ]:
vector_df=pd.DataFrame(vectors,columns=range(0,5000))


In [ ]:
vector_df.shape

In [ ]:
vector_df.head()

In [ ]:
import h5py

In [ ]:

def compute(vector_df,batch_size=1000,output_file='similarity_matrix3.h5'):
    num_samples=vector_df.shape[0]
    with h5py.File(output_file,'w') as f:
        dset=f.create_dataset("similarity_matrix3",shape=(num_samples,num_samples),dtype=np.float32)
        for i in range(0,num_samples,batch_size):
          batch=vector_df[i:i+batch_size]
          similarity=cosine_similarity(batch,vector_df)
          dset[i:i+batch_size,:]=similarity 
          print(f"Proceesed batch{i} to {i+batch_size}")

similarity_matrix=compute(vector_df,batch_size=1000,output_file='similarity_matrix3.h5')

